#### ¿Dónde está alojada la base de datos?

La base **SQLite** está alojada localmente en un archivo llamado `mi_base.sqlite`.  
No requiere servidor: todo ocurre de forma local.

---

#### ¿Qué motor de base de datos se utiliza?

Se utiliza **SQLite**, un motor ligero ideal para bases de datos **pequeñas y medianas**.  
Es embebido, no necesita configuración de servidor ni instalación adicional.

---

#### ¿Cómo nos conectamos a la base de datos?

Se utiliza el módulo `sqlite3` de Python (incluido en la librería estándar).  
La conexión se realiza con:

```python
conn = sqlite3.connect("mi_base.sqlite")

conn.execute(...)         # Para comandos directos
pd.read_sql(query, conn)  # Para cargar resultados como DataFrame

In [ ]:
import sqlite3
import pandas as pd

# Mostrar todas las columnas al imprimir
pd.set_option('display.max_columns', None)

# Leer CSV con pandas
df = pd.read_csv("./archive/alquiler_02_11_2023cc.csv")

# Conectar (o crear) base SQLite local
conn = sqlite3.connect("mi_base.sqlite")

# Cargar el DataFrame a una tabla llamada 'alquileres'
df.to_sql('alquileres', conn, if_exists='replace', index=False)

# Consultar las primeras 5 filas con SQL
query = "SELECT * FROM alquileres LIMIT 5;"
df_head = pd.read_sql(query, conn)

print(df_head)


Realizamos una observación del tipo de información de hay en cada columna

In [ ]:
info_query = "PRAGMA table_info(alquileres);"
df_info = pd.read_sql(info_query, conn)

print(df_info)

Seleccionamos las columnas con las que nos interesaría trabajar

In [ ]:
query = """
SELECT titulo, precio, direction, superficie_total, superficie_util,
       ambientes, dormitorios, banos, tipo_departamento, comuna, region, estacionamientos, bodegas
FROM alquileres;
"""

df_filtrado = pd.read_sql(query, conn)
print(df_filtrado.head())

Realizamos un análisis porcentual de datos faltantes por columna, teniendo en cuenta el siguiente criterio:


| % de valores faltantes | Acción                    |
|------------------------|-----------------------------------------------------|
| > 50%                  | Eliminar la columna                                 |
| 20% - 50%              | Evaluar si imputar o conservar según relevancia     |
| < 20%                  | Imputar los valores faltantes                       |
| 0% - 5%                | No es preocupante |


In [ ]:
filas_totales = df_filtrado.shape[0]
porcentaje_faltante_columna = (df_filtrado.isna().sum() / filas_totales) * 100
print(porcentaje_faltante_columna.round(2).astype(str) + " %")


Por lo tanto, eliminamos las columnas tipo_departamento, estacionamientos y bodegas

In [ ]:
query = """
SELECT titulo, precio, direction, superficie_total, superficie_util,
       ambientes, dormitorios, banos, comuna, region
FROM alquileres;
"""

df_filtrado = pd.read_sql(query, conn)
print(df_filtrado.head())

Procedemos a hacer un análisis de outliers en la variable "precio", para esto calculamos el primer cuartil (Q1) y el tercer cuartil (Q3) de la distribución de precios, y a partir de ellos obtenemos el rango intercuartílico (IQR = Q3 - Q1).

Luego, extraemos las filas donde el precio está fuera de estos límites (por debajo del límite inferior o por encima del superior) para identificar los valores extremos.

In [ ]:
# Calcular Q1 y Q3 de la columna 'precio'
q1 = df_filtrado['precio'].quantile(0.25)
q3 = df_filtrado['precio'].quantile(0.75)
iqr = q3 - q1

# Calcular límites para detectar outliers
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

print(f"Límite inferior: {lower_bound}, Límite superior: {upper_bound}")

query = f"""
SELECT *
FROM alquileres
WHERE precio < {lower_bound} OR precio > {upper_bound}
"""
outliers_df = pd.read_sql(query, conn)

Grafico el IQR (que representa la dispersión central del conjunto de datos):

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
sns.boxplot(x=df_filtrado['precio'])
plt.title('Boxplot de Precio')
plt.xlabel('Precio')
plt.show()

Comparamos la data previamente identificada como outliers (outliers_df) con el resto del conjunto de datos (df_filtrado) para observar el comportamiento y distribución de dichos valores atípicos en relación a la variable precio y la superficie_total.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_filtrado, x='superficie_total', y='precio', label='Datos normales', alpha=0.5)
sns.scatterplot(data=outliers_df, x='superficie_total', y='precio', color='red', label='Outliers')
plt.title('Comparación entre precios normales y outliers')
plt.legend()
plt.grid(True)
plt.show()


En primer lugar, observamos outliers con precios elevados pero con superficie total igual a cero, además de casos con precios de alquiler iguales a cero, lo cual es incoherente y podría afectar nuestro análisis posterior.

In [ ]:
query = """
SELECT COUNT(*) AS cantidad
FROM alquileres
WHERE precio = 0
   OR superficie_total = 0
   OR superficie_util = 0
"""
resultado = pd.read_sql(query, conn)
print(f"Cantidad de filas con precio o superficie = 0: {resultado['cantidad'][0]}")



Borramos las filas con precio o superficie = 0

In [ ]:
query_delete = """
DELETE FROM alquileres
WHERE precio = 0
   OR superficie_total = 0
   OR superficie_util = 0
"""
conn.execute(query_delete)
conn.commit()


Análizamos un nuevo caso ilógico: El hecho de que haya alquileres cuya superficie total sea menor a su superficie útil

In [ ]:
# Total de filas
total_filas = pd.read_sql("SELECT COUNT(*) AS total FROM alquileres", conn)['total'][0]

# Filas con inconsistencia: superficie_total < superficie_util
query = """
SELECT COUNT(*) AS cantidad
FROM alquileres
WHERE superficie_total < superficie_util
"""
resultado = pd.read_sql(query, conn)
cantidad = resultado['cantidad'][0]

# Porcentaje
porcentaje = (cantidad / total_filas) * 100
print(f"Cantidad de filas con superficie_total < superficie_util: {cantidad}")
print(f"Porcentaje respecto al total: {porcentaje:.2f} %")


Eliminamos dichas filas (que representan un 1.46% del total de los datos)

In [ ]:
delete_query = """
DELETE FROM alquileres
WHERE superficie_total < superficie_util
"""

conn.execute(delete_query)
conn.commit()  # Asegura que se guarden los cambios

LIMPIEZAS OPCIONALES: A) Un alquiler sin baños no es habitable ni se le puede brindar un uso doméstico o de oficina, ocurre lo mismo para ambientes no pueden ser menores que 1

In [ ]:
query_delete_opt = "DELETE FROM alquileres WHERE banos = 0 OR ambientes < 1"
conn.execute(query_delete_opt)
conn.commit()   

B) Otro punto a tener en cuenta es la columna "cantidad_max_habitantes" la cual cuenta con muchas filas en 0, lo cuál lo hace algo ilógico. Si bien no es una columna con la que no se decidio trabajar no tiene sentido que ningun habitante pueda estar en el alquiler
PD: en cuanto a la columna dormitorios se sobre entiende que si no tiene dormitorios es un monoambiente,por lo tanto, debería validarse esta relacion

In [ ]:
query_delete = "DELETE FROM alquileres WHERE cantidad_max_habitantes < 1 OR (dormitorios = 0 AND ambientes > 1)"
conn.execute(query_delete)
conn.commit()

Nuevamente, ya con los datos limpios, volvemos a analizar con un gráfico los resultados

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_filtrado, x='superficie_total', y='precio', label='Datos normales', alpha=0.5)
sns.scatterplot(data=outliers_df, x='superficie_total', y='precio', color='red', label='Outliers')
plt.title('Comparación entre precios normales y outliers')
plt.legend()
plt.grid(True)
plt.show()